## Welcome !!
#### As you must have guessed from the name of the notebook, this is an implementation from scratch of the UNET model , that was earlier developed for biomedical image segmentation and localization of the diseases. However, now with some modifications it is used in various other tasks like object detection, semantic segementation and also noise removal from images(by training the model for it).

#### But in this notebook i have tried to build the UNET model by keeping it's architecture as close as possible to the original paper architecture.Please show some support by upvoting the notebook if you like it.

#### You can read the paper from the following link https://arxiv.org/pdf/1505.04597


In [ ]:
# importing the necessary libraries
import numpy as np
import pandas as pd
import os
import cv2
from tensorflow.keras.utils import plot_model
from keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D,Activation,MaxPooling2D,BatchNormalization,Conv2DTranspose,CenterCrop,Concatenate,Input
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import tensorflow

In [ ]:
# preparing the dataset
image_path=[]
mask_path=[]
df=pd.DataFrame()
root_path='/kaggle/input/lgg-mri-segmentation/kaggle_3m'
for folder in os.listdir(root_path):
    if folder.startswith('T'):
        for path in os.listdir(root_path+'/'+folder):
            if 'mask' in path:
                mask_path.append(root_path+'/'+folder+'/'+path)
            else:
                image_path.append(root_path+'/'+folder+'/'+path)
                
    elif folder=='data.csv':
        df=pd.read_csv(root_path+'/'+folder)
    else: 
        continue

In [ ]:
df.head()

In [ ]:
# image_id=[]
# mask_id=[]
# for id_ in df['Patient']:
#     for path in image_path:
#         if id_ in path:
#             image_id.append(id_)
#     for path in mask_path:
#         if id_ in path:
#             mask_id.append(id_)

In [ ]:
patient_id=[id_[67:-4] for id_ in image_path]

In [ ]:
def filter(mask_path):
    if np.max(cv2.imread(mask_path))>0.5:
        return 1
    else:
        return 0
df_brain=pd.DataFrame({'Patient_id':patient_id,'Image_path':image_path,'Mask_path':mask_path})
df_brain['Diagnosis']=df_brain['Mask_path'].apply(lambda x: filter(x))

In [ ]:
df_brain.head()

In [ ]:
df_brain.drop('Patient_id',axis=1,inplace=True)

In [ ]:
df_brain_train,df_brain_test=train_test_split(df_brain,test_size=0.2)

In [ ]:
# def get_data(img_path,mask_path):
#     x_train=[]
#     y_train=[]
#     for i in range(len(img_path)):
#         img=cv2.imread(img_path[i])
#         img=cv2.resize(img,(256,256))
#         mask=cv2.imread(mask_path[i],cv2.IMREAD_GRAYSCALE)
#         mask=cv2.resize(mask,(255,255))
#         mask=np.expand_dims(mask,axis=-1)
#         mask=mask/256
#         x_train.append(img)
#         y_train.append(mask)
#     return x_train,y_train

In [ ]:
# Augmenting data
def get_augmented_data(df,aug_dict,batch_size=32,image_size=(256,256)):
    img_gen=ImageDataGenerator(**aug_dict)
    mask_gen=ImageDataGenerator(**aug_dict)
    image_gen=img_gen.flow_from_dataframe(df,x_col='Image_path',target_size=image_size,batch_size=batch_size,color_mode='rgb',class_mode=None)
    mask_gen=mask_gen.flow_from_dataframe(df,x_col='Mask_path',target_size=image_size,batch_size=batch_size,color_mode='grayscale',class_mode=None)
    for img,mask in zip(image_gen,mask_gen):
        img=img/255
        mask=mask/255
        mask[mask>0.5]=1
        mask[mask<=0.5]=0
        yield(img,mask)
aug_dict=dict(rotation_range=0.2,
                            width_shift_range=0.05,
                            height_shift_range=0.05,
                            shear_range=0.05,
                            zoom_range=0.05,
                            horizontal_flip=True,
                            fill_mode='nearest')
train_gen=get_augmented_data(df_brain_train,aug_dict)
test_gen=get_augmented_data(df_brain_test,aug_dict={})

### We have to create out own loss function to calculate the loss between the original and the predicted mask, so bce_dice loss is the loss function that is used for that reason.

In [ ]:
# Loss function
def bce_loss(y_true,y_pred):
    y_true=K.cast(y_true,'float32')
    y_pred=K.cast(y_pred,'float32')
    return tensorflow.keras.losses.binary_crossentropy(y_true,y_pred)
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    
    y_true_f = K.cast(y_true_f, 'float32')
    y_pred_f = K.cast(y_pred_f, 'float32')
    
    intersection = K.sum(y_true_f * y_pred_f)
    dice_coef_v = (2. * intersection + 1.) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.)
    return dice_coef_v
def dice_loss(y_true, y_pred):
    dice_loss_v = 1 - dice_coef(y_true, y_pred)
    return dice_loss_v
def bce_dice_loss(y_true, y_pred):
    bce_dice_loss_v = bce_loss(y_true, y_pred) + dice_loss(y_true, y_pred)
    return bce_dice_loss_v

### Now we will create a convolution block that will perform two convolution operations without any maxpooling in between with relu as the activation function. At last we perform the max pooling2D except if it is the last block after which we have to perform the up-convolution.

In [ ]:
def convolution(x,num_filters,last_block=False):
    x=Conv2D(num_filters,(3,3),padding='same')(x)
    x=Activation('relu')(x)
    x=Conv2D(num_filters,(3,3),padding='same')(x)
#     bn=BatchNormalization(conv)
    x=Activation('relu')(x)
    skip=x
    if not last_block:
        x=MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)
    return x,skip
#     return x

### Next we create build the contracting path of the model, as shown in the paper.

In [ ]:
starting_filters=64
def calculate_filters(idx):
    return (starting_filters*(2**idx))

def contracting_path(x,num_unet_blocks=5):
#     x=0
    skip_connections=[]
    for i in range(num_unet_blocks):
        filters=calculate_filters(i)
#         filters=64
        last_block=i==num_unet_blocks-1
        if last_block:
            x,skip_input=convolution(x,filters,last_block=last_block)
        else:    
            x,skip_input=convolution(x,filters,last_block=last_block)
            skip_connections.append(skip_input)
    return x,skip_connections

### Next we create the expanding path by performing Conv2dTranspose.

In [ ]:
def upconv(x,num_filters,skip_input,last_block=False):
    x=Conv2DTranspose(num_filters,(2,2),strides=(2,2))(x)
    if not last_block:
        skip_crop_input=CenterCrop(height=x.shape[1],width=x.shape[2])(skip_input)
        x=Concatenate(axis=-1)([skip_crop_input,x])
    x=Conv2D(num_filters,(3,3),padding='same')(x)
    x=Activation('relu')(x)
    x=Conv2D(num_filters,(3,3),padding='same')(x)
    x=Activation('relu')(x)
    return x
def expanding_path(x,skip_connections,num_unet_blocks=4):
    for i in range(num_unet_blocks):
        filters=calculate_filters(num_unet_blocks-i-1)
        x=upconv(x,filters,skip_connections[num_unet_blocks-i-1])
    return x

In [ ]:
# Creaing the unet model
def create_unet():
    input_data=Input(shape=(256,256,3))
    latents,skip_connections=contracting_path(input_data)
    expanded_data=expanding_path(latents,skip_connections)
    final_output=Conv2D(1,(1,1),activation='sigmoid')(expanded_data)
    model=Model(inputs=[input_data],outputs=[final_output])
    return model

In [ ]:
# def init_model():
model=create_unet()
loss_init=bce_dice_loss
metrics=[dice_coef]
epochs=32
model.compile(loss=loss_init,metrics=metrics)

In [ ]:
model.summary()

In [ ]:
plot_model(model,to_file='unet.png')

In [ ]:
# Applying model checkpointing and early stopping
checkpoint=ModelCheckpoint('best_unet_model.h5',
                           monitor='val_loss',
                           mode='min',
                           verbose=1,
                           save_best_only=True)
early_stopping=EarlyStopping(monitor='val_loss',
                             patience=15,
                             mode='min',
                             verbose=1)

In [ ]:
# Training the model
epochs=100
batch_size=32
history=model.fit(train_gen,
                  epochs=epochs,
                  steps_per_epoch=len(df_brain_train)//batch_size,
                  validation_data=test_gen,
                  validation_steps=len(df_brain_test)//batch_size,
                 callbacks=[early_stopping,checkpoint])


## The model weights can be downloaded from the working directory of the kernal. Comment if you are unable to do so. Also, comment for any kind of problems related to this kernal.